<a href="https://colab.research.google.com/github/loomdawg/PHYS450Labs/blob/main/phys450lab2data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import libraries
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-deep')
import pandas as pd
import uncertainties.umath as u
from uncertainties import ufloat
from uncertainties.unumpy import uarray
from uncertainties import nominal_value as nv
from uncertainties import std_dev as sd

2. Focal Length from Lens Geometry

In [ ]:
# function to calculate Radius
def calculate_curvature(h, s): #returns radius R
  return (h**2 + s**2) / (2 * h)

In [ ]:
#function to calculate focal length from radius
def manual_calculate_f_thick(R_1, R_2, t, n = 1.517): #returns focal length
  one_over_f = (n - 1) * ((1 / R_1) - (1 / R_2) + ((n - 1) * t) / (n * R_1 * R_2))
  return 1 / one_over_f

def manual_calculate_f_thin(R_1, R_2, n = 1.517): #returns focal length
  one_over_f = (n - 1) * ((1 / R_1) - (1 / R_2))
  return 1 / one_over_f



In [ ]:
# input data and calculate average values #everything is measured in mm

#input data
# h_0_array = [19.940, 19.939, 19.909] 
# h_0_array = [19.940, 19.939]
# s_array = [19.76, 19.51, 19.61]

# h_2_up_array = [19.234, 19.239, 19.234] 
# h_2_down_array = [19.231, 19.340, 19.320] #DL02 with letts down 
# h_2_down_array = uarray([19.231, 19.340, 19.320], [.01, .01, .01])

h_1_up_array = [20.801, 20.799, 20.802] #CL02 with letters up
h_1_down_array = [20.805, 20.803, 20.801] #CL02 with letters down #data on CL02 is much better than data on DL02

h_0_array = uarray([19.940, 19.939], [.01, .01]) #the last value in this was definitely an outlier.  It is very likely that it was a bad measurement, so we'll remove it
s_array = uarray([19.76, 19.51, 19.61], [.1, .1, .1])
h_2_up_array = uarray([19.234, 19.239, 19.234], [.01, .01, .01]) #DL02 with letters up
h_2_down_array = uarray([19.340, 19.320], [.1, .1]) #its hard to say if the first value in this array is an outlier, we'll keep it for now but this will really screw up the error analysis
h_1_up_array = uarray([20.801, 20.799, 20.802], [.01, .01, .01])
h_1_down_array = uarray([20.805, 20.803, 20.801],[.01, .01, .01])

t_c = ufloat(10.0, 1)
t_d = ufloat(2.0, 1)

# average values just to get a feel for all the measurements
h_0 = h_0_array.mean()

h_1_up = h_1_up_array.mean()
h_1_down = h_1_down_array.mean()

h_2_up = h_2_up_array.mean()
h_2_down = h_2_down_array.mean()

h_c_up = h_0 - h_1_up
h_c_down = h_0 - h_1_down
h_d_up = h_0 - h_2_up
h_d_down = h_0 - h_2_down

s = s_array.mean()

# print data values just to see where we're at # I tried writing a function for this, but no luck.  
print('h_0 = {h_0}\n<s> = {s}\nh_c_up = {hcup}\nh_c_down = {hcdown}\nh_d_up = {hdup}\nh_d_down = {hddown}'.format(h_0 = h_0, s = s, hcup = h_c_up, hcdown = h_c_down, hdup = h_d_up, hddown = h_d_down))




h_0 = 19.940+/-0.007
<s> = 19.63+/-0.06
h_c_up = -0.861+/-0.009
h_c_down = -0.863+/-0.009
h_d_up = 0.704+/-0.009
h_d_down = 0.61+/-0.07


In [ ]:
# lets make a method to tie everything together
def calculate_f_thick(h_0_array, h_1_array, h_2_array, s_array, t): #input the raw data, returns focal length, R_1, R_2
  h_0 = h_0_array.mean()
  h_1 = h_1_array.mean()
  h_2 = h_2_array.mean()
  s = s_array.mean()

  h_up = h_0 - h_1
  h_down = h_0 - h_2

  R_up = -calculate_curvature(h_up, s)
  R_down = calculate_curvature(h_down, s)

  return (manual_calculate_f_thick(R_up, R_down, t), R_up, R_down)

def calculate_f_thin(h_0_array, h_1_array, h_2_array, s_array): #input raw data, returns focal length, R_1, R_2
  h_0 = h_0_array.mean()
  h_1 = h_1_array.mean()
  h_2 = h_2_array.mean()
  s = s_array.mean()

  h_up = h_0 - h_1
  h_down = h_0 - h_2

  R_up = -calculate_curvature(h_up, s)
  R_down = calculate_curvature(h_down, s)

  return (manual_calculate_f_thin(R_up, R_down), R_up, R_down)

In [ ]:
# calculate focal lengths
f_c_thick, R_c_up, R_c_down = calculate_f_thick(h_0_array, h_1_up_array, h_1_down_array, s_array, t_c)
f_c_thin, R_c_up, R_c_down = calculate_f_thin(h_0_array, h_1_up_array, h_1_down_array, s_array)

f_d_thick, R_d_up, R_d_down = calculate_f_thick(h_0_array, h_2_up_array, h_2_down_array, s_array, t_d)
f_d_thin, R_d_up, R_d_down = calculate_f_thin(h_0_array, h_2_up_array, h_2_down_array, s_array)

#save values
data = {
    'converging':[nv(f_c_thick), nv(f_c_thin), nv(R_c_up), nv(R_c_down)],
    '+- c_error':[sd(f_c_thick),sd(f_c_thin), sd(R_c_up), sd(R_c_down)],
    'diverging': [nv(f_d_thick), nv(f_d_thin), nv(R_d_up), nv(R_d_down)],
    '+- d_error':[sd(f_d_thick), sd(f_d_thin), sd(R_d_up), sd(R_d_down)],
}

# put values into df
df = pd.DataFrame(data, index = ['thick f (mm)', 'thin f (mm)', 'R_up (mm)', 'R_down (mm)'])
df.style.set_caption('Focal Lengths (thick and thin approximation) and Curvature Calculations from Lens Geometry')

#display df

# print(df)

df

,converging,+- c_error,diverging,+- d_error
thick f (mm),218.084404,2.410242,-283.654584,15.707019
thin f (mm),216.423772,2.404568,-283.982111,15.703558
R_up (mm),224.084180,2.707499,-274.000539,3.893143
R_down (mm),-223.480994,2.694590,316.306427,36.854931


Focal Length Using Parallel Rays